# Getting started with Cartopy

[**cartopy**](https://scitools.org.uk/cartopy/docs/latest/) is a Python package designed for geospatial data processing in order to produce maps and other geospatial data analyses. It makes use of the powerful ```PROJ```, ```NumPy``` and ```Shapely``` libraries and includes a programmatic interface built on top of ```matplotlib``` for the creation of publication quality maps.

First of all, let's import all the required Python modules

In [ ]:
%matplotlib inline
import cartopy.crs as ccrs
from cartopy.mpl.geoaxes import GeoAxes
from cartopy.util import add_cyclic_point
import numpy as np
import pandas as pd
import xarray as xr
import matplotlib.pyplot as plt
import matplotlib as mpl
import glob
import warnings
warnings.filterwarnings("ignore")
from os.path import expanduser
home = expanduser("~")

All the input datasets are located under the ```data``` folder and organized according to the directory structure defined in the [**CMIP6 Data Reference Syntax**](https://pcmdi.llnl.gov/CMIP6/Guide/dataUsers.html). 

In this notebook we are going to use a ```tas```(near-surface air temperature) file which is part of the ```CMIP6``` dataset produced by ```CMCC``` from the ```CMCC-CM2-SR5``` global coupled general circulation model and related to the ```ssp585``` experiment (update of emission-driven RCP8.5 based on SSP5). 

In [ ]:
input_file = home+"/data/CMIP6/ScenarioMIP/CMCC/CMCC-CM2-SR5/ssp585/r1i1p1f1/Amon/tas/gn/v20200622/tas_Amon_CMCC-CM2-SR5_ssp585_r1i1p1f1_gn_201501-210012.nc"
ds = xr.open_dataset(input_file).load()
ds

### Plotting a Two-dimensional Field

As a first example, we want to plot on a map the near-surface air temperature at a specific time point in the ```PlateCarree``` projection **without** specifying the ```transform``` argument. We can select the timestep by its index (e.g. ```time_index = 0``` to get ```January 2015```).

Since data happen to be defined in the same coordinate system as we are plotting in, this actually works correctly.

In [ ]:
fig = plt.figure(figsize=(10, 5), dpi=100)

#Add Geo axes to the figure with the specified projection (PlateCarree)
projection = ccrs.PlateCarree()
ax = plt.axes(projection=projection)

#Draw coastline and gridlines
ax.coastlines()

gl = ax.gridlines(crs=projection, draw_labels=True, linewidth=1, color='black', alpha=0.9, linestyle=':')
gl.xlabels_top = False
gl.ylabels_right = False

#Get the near-surface air temperature field and the dimensions values
time_index = 0
tas = ds.tas.isel(time=time_index)
lat = ds.lat
lon = ds.lon
tas = np.reshape(tas, (len(lat), len(lon)))

#Wraparound points in longitude
var_cyclic, lon_cyclic = add_cyclic_point(tas, coord=np.asarray(lon))
x, y = np.meshgrid(lon_cyclic,lat)

#Define color levels for color bar
levStep = (np.nanmax(tas)-np.nanmin(tas))/20
clevs = np.arange(np.nanmin(tas),np.nanmax(tas)+levStep,levStep)

#Set filled contour plot
cnplot = ax.contourf(x, y, var_cyclic, clevs,cmap=plt.cm.jet)  # didn't use transform, but looks ok...
plt.colorbar(cnplot,ax=ax)

ax.set_aspect('auto', adjustable=None)

plt.title('Near-Surface Air Temperature (deg K) - '+str(pd.to_datetime(ds.time.values[time_index].strftime("%Y%m%d %H%M%S"))))
plt.show()

Now let’s add in the ```transform``` keyword in the ```contourf``` method when we plot.

Note that the plot doesn’t change. This is because the default assumption when the ```transform``` argument **is not supplied** is that the coordinate system matches the projection, which has been the case so far.

In [ ]:
fig = plt.figure(figsize=(10, 5), dpi=100)

#Add Geo axes to the figure with the specified projection (PlateCarree)
projection = ccrs.PlateCarree()
ax = plt.axes(projection=projection)

#Draw coastline and gridlines
ax.coastlines()

gl = ax.gridlines(crs=projection, draw_labels=True, linewidth=1, color='black', alpha=0.9, linestyle=':')
gl.xlabels_top = False
gl.ylabels_right = False

#Get the near-surface air temperature field and the dimensions values
time_index = 0
tas = ds.tas.isel(time=time_index)
lat = ds.lat
lon = ds.lon
tas = np.reshape(tas, (len(lat), len(lon)))

#Wraparound points in longitude
var_cyclic, lon_cyclic = add_cyclic_point(tas, coord=np.asarray(lon))
x, y = np.meshgrid(lon_cyclic,lat)

#Define color levels for color bar
levStep = (np.nanmax(tas)-np.nanmin(tas))/20
clevs = np.arange(np.nanmin(tas),np.nanmax(tas)+levStep,levStep)

#Set filled contour plot
cnplot = ax.contourf(x, y, var_cyclic, clevs,transform=projection, cmap=plt.cm.jet)  # transform keyword provided
plt.colorbar(cnplot,ax=ax)

ax.set_aspect('auto', adjustable=None)

plt.title('Near-Surface Air Temperature (deg K) - '+str(pd.to_datetime(ds.time.values[time_index].strftime("%Y%m%d %H%M%S"))))
plt.show()

Now we’ll try again using a different projection for our plot. We’ll plot onto an ```Orthographic``` projection, with ```central_longitude=35.0``` and ```central_latitude=35.0```. 

In [ ]:
time_index = 0
p = ds.tas.isel(time=time_index).plot(
    subplot_kws=dict(projection=ccrs.Orthographic(35, 35), facecolor="gray"),
    transform=ccrs.PlateCarree(),
)
p.axes.set_global()
p.axes.coastlines()

And the same plot with different ```central_longitude``` and ```central_latitude```. 

In [ ]:
p = ds.tas.isel(time=time_index).plot(
    subplot_kws=dict(projection=ccrs.Orthographic(-90, 50), facecolor="gray"),
    transform=ccrs.PlateCarree(),
)
p.axes.set_global()
p.axes.coastlines()

### Faceted maps

We can make faceted maps. Since ```FacetGrid``` creates the axes it plots to, we need to pass the ```projection``` kwarg in ```subplot_kws```. This makes sure that the subplots are set up properly for cartopy.

In [ ]:
monthly_means = ds.groupby("time.month").mean()
monthly_means.tas.attrs = ds.tas.attrs

fg = monthly_means.tas.isel(month=[1, 2, 3]).plot(
    col="month",
    transform=ccrs.PlateCarree(),
    subplot_kws={
        "projection":ccrs.PlateCarree()
    },
    cbar_kwargs={"orientation": "horizontal", "shrink": 0.8, "aspect": 40},
    robust=True,
)

fg.map(lambda: plt.gca().coastlines())

When faceting on maps, the projection can be transferred to the plot function using the ```subplot_kws``` keyword. 

The axes for the subplots created by faceting are accessible in the object returned by ```plot```.

This is the plot with the ```LambertConformal``` projection...

In [ ]:
monthly_means = ds.groupby("time.month").mean()
monthly_means.tas.attrs = ds.tas.attrs

fg = monthly_means.tas.isel(month=[1, 2, 3]).plot(
    col="month",
    transform=ccrs.PlateCarree(),
    subplot_kws={
        "projection": ccrs.LambertConformal(
            central_longitude=35, central_latitude=35
        )
    },
    cbar_kwargs={"orientation": "horizontal", "shrink": 0.8, "aspect": 40},
    robust=True,
)

fg.map(lambda: plt.gca().coastlines())

... and with the ```Orthographic``` projection.

In [ ]:
monthly_means = ds.groupby("time.month").mean()
monthly_means.tas.attrs = ds.tas.attrs

fg = monthly_means.tas.isel(month=[1, 2, 3]).plot(
    col="month",
    transform=ccrs.PlateCarree(),
    subplot_kws={"projection": ccrs.Orthographic(35, 35)},
    cbar_kwargs={"orientation": "horizontal", "shrink": 0.8, "aspect": 40},
    robust=True,
)

fg.map(lambda: plt.gca().coastlines())
fg.map(lambda: plt.gca().gridlines())